In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
# 단어장(vocabulary) 만들기
vocab = list(set(sentence))
print(vocab)

['the', 'best', 'is', 'for', 'memory', 'medicine', 'Repeat']


In [4]:
# 단어장의 단어에 고유한 정수 인덱스 부여 // 모르는 단어 의미하는 UNK 토큰 추가
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)

{'the': 1, 'best': 2, 'is': 3, 'for': 4, 'memory': 5, 'medicine': 6, 'Repeat': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

5


In [7]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'the', 2: 'best', 3: 'is', 4: 'for', 5: 'memory', 6: 'medicine', 7: 'Repeat', 0: '<unk>'}


In [8]:
print(index2word[2])

best


In [9]:
# 정수 2 와 맵핑되는 단어는 Repeat
# 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수 생성

def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [10]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[7, 3, 1, 2, 6, 4]])
tensor([[3, 1, 2, 6, 4, 5]])


In [11]:
# 모델 구현
# 이전 모델들과는 임베딩 층을 추가 함으로써 차별점을 둠
# 임베딩 층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기, 두번째 인자는 임베딩 백터의 차원


class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                           embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                               batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) #출력은 원-핫 벡터 크기를 가져야 함 // 또는 집합 크기만큼 가져야함
        
    def forward(self, x):
        # 1. 임베딩 층
        # 크기 변화 : (배치 크기, 시퀸스 길이) => (배치 크기, 시퀸스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기 변화 : (배치 큭, 시퀸스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀸스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기 변화 : (배치 크기, 시퀸스 길이, 은닉층 크기) => (배치 크기, 시퀸스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기 변화 : (배치 크기, 시퀸스 길이, 단어장 크기) => (배치 크기*시퀸스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [12]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용 // unk 토큰을 크기에 포함
input_size = 5 # 임베딩 된 차원으 크기 및 RNN 층 입력 차원의 크기 5
hidden_size = 20 # RNN 은닉층 크기 20

In [13]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실 함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제 값은 원-핫 인코딩 안해도 됨

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [14]:
# 모델에 입력을 넣어 출력 확인
output = model(X)
print(output)

tensor([[-0.0504, -0.0210, -0.0638, -0.0544, -0.0907,  0.0745,  0.2469, -0.0269],
        [-0.0683, -0.1977, -0.0609, -0.1113, -0.0045,  0.2532,  0.1841, -0.0450],
        [-0.0396, -0.1596, -0.1126, -0.0604,  0.0013,  0.2629,  0.2141,  0.0188],
        [-0.3039,  0.1228,  0.1537,  0.1477, -0.0006, -0.3376,  0.5612,  0.0191],
        [-0.1026, -0.2360,  0.1603,  0.1062, -0.0864,  0.1427,  0.0903, -0.0942],
        [-0.2135, -0.0471,  0.2249,  0.1795, -0.1239, -0.1268, -0.0963, -0.1290]],
       grad_fn=<ViewBackward0>)


In [15]:
# 예측 값의 크기는 (6, 8)
print(output.shape)

torch.Size([6, 8])


In [16]:
# 모델 훈련 전에 예측을 제대로 하고 있는지 예측된 정수 시퀸스로 바꾸는 decode 함수 생성
decode = lambda y: [index2word.get(x) for x in y]

In [17]:
# 훈련 시작 에포크 200
for step in range(201):
    #경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201]{:4f}".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201]2.098062
Repeat medicine memory memory medicine best best

[41/201]1.663813
Repeat medicine the memory medicine for memory

[81/201]1.125715
Repeat is the best medicine for memory

[121/201]0.557082
Repeat is the best medicine for memory

[161/201]0.255167
Repeat is the best medicine for memory

[201/201]0.139648
Repeat is the best medicine for memory

